# Experimenting With APIs

In this notebook, we will be experimenting with the APIs and create helper functions.

In [1]:
from tqdm.auto import tqdm
import itertools

import os

# my imports
from helpers import kube
from helpers import workload
from helpers import util
from helpers import request_funcs

# experiment utility functionality used in experiments
from exp_util import *

starting watch thread...


## Kubernetes API

We use the kubernetes API to fetch the current status of each deployment and see how many instances they have.

In [2]:
# print details of a deployment
get_knative_watch_info('bench1')

{'kind': 'Deployment',
 'replicas': 0,
 'ready_replicas': 0,
 'last_update': 1615048672.3186128}

## Deploying On Knative

In this section, we will develop the functionality to deploy a given workload to **knative** using the `kn` CLI.
The other options were using [CRDs](https://stackoverflow.com/questions/61384188/how-to-deploy-a-knative-service-with-kubernetes-python-client-library),
or using [kubectl apply](https://developers.redhat.com/coderland/serverless/deploying-serverless-knative#invoking_your_service_from_the_command_line)
but `kn` seems to be more powerful.

In [3]:
# making sure kn is set up correctly
!kn service ls

NAME            URL                                            LATEST                AGE   CONDITIONS   READY   REASON
autoscale-go    http://autoscale-go.default.kn.nima-dev.com    autoscale-go-00023    33d   3 OK / 3     True    
bench1          http://bench1.default.kn.nima-dev.com          bench1-00043          42d   3 OK / 3     True    
helloworld-go   http://helloworld-go.default.kn.nima-dev.com   helloworld-go-00001   43d   3 OK / 3     True    


In [4]:
EXP_CONFIG_NAME_DEFAULT = 'bench1_sleep_rand2_1000_200'
exp_config_name = os.getenv("EXP_CONFIG_NAME", EXP_CONFIG_NAME_DEFAULT)
exp_file = f"configs/{exp_config_name}.json"

workload_spec = util.load_json_file(exp_file)

kn_command = kube.get_kn_command(**workload_spec)
print(kn_command)
# to run the command, we can simply use:
# !{kn_command}

kn service apply autoscale-go --image gcr.io/knative-samples/autoscale-go:0.1 \
   \
  --limit 'cpu=250m,memory=256Mi' \
  -a autoscaling.knative.dev/panicThresholdPercentage=1000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency


# Workload Specification

In [5]:
# get user workload function from the file
user_workload_func = request_funcs.workload_funcs[workload_spec['request_func']]
# get ready count callback
get_ready_cb = lambda: get_knative_watch_info(workload_spec['name'])['ready_replicas']
print('ready callback:', get_ready_cb())
# create logger and check one execution of workload func
wlogger = workload.WorkloadLogger(get_ready_cb=get_ready_cb)
simple_worker_func = lambda: wlogger.worker_func(user_workload_func)
# add worker func to workload spec
workload_spec['simple_worker_func'] = simple_worker_func

simple_worker_func()

ready callback: 0


{'client_start_time': 1615048673.5082343,
 'client_end_time': 1615048676.4656272,
 'client_elapsed_time': 2.957392930984497,
 'start_conc': 1,
 'end_conc': 1,
 'success': True,
 'start_ready_count': 0,
 'end_ready_count': 0}

In [6]:
# wlogger.monitoring_thread.start()
# wlogger.record_conc_loop()
# wlogger.monitor_conc_loop()
wlogger.start_capturing()
time.sleep(7)
wlogger.stop_capturing()
wlogger.get_recorded_data()

starting threads
stopping threads...
Done.


{'ready_count': [0, 0, 1, 1],
 'total_conc': [0, 0, 0, 0],
 'conc_window_average': [0.0, 0.0, 0.0, 0.0],
 'time': [1615048676.478768,
  1615048678.4827847,
  1615048680.49134,
  1615048682.5001976]}

# Performing A Series of Experiments

In [7]:
target_list = workload_spec['exp_spec']['target_list']
rps_list = workload_spec['exp_spec']['rps_list']

In [8]:
overall_results = []
total_configs = list(itertools.product(target_list, rps_list))
for target, rps in tqdm(total_configs):
    single_res_name = perform_experiment(rps=rps, target=target, base_workload_spec=workload_spec, wlogger=wlogger)
    overall_results.append({
        'target': target,
        'rps': rps,
        'res_name': single_res_name,
        'request_func': workload_spec['request_func'],
        'workload_name': workload_spec['name'],
        'is_rps': workload_spec['is_rps'],
        'exp_time_mins': workload_spec['exp_spec']['time_mins'],
    })


  0%|          | 0/48 [00:00<?, ?it/s]

applying kn command:
kn service apply autoscale-go --image gcr.io/knative-samples/autoscale-go:0.1 \
   \
  --limit 'cpu=250m,memory=256Mi' \
  -a autoscaling.knative.dev/panicThresholdPercentage=1000 \
  -a autoscaling.knative.dev/target=1 \
  -a autoscaling.knative.dev/metric=concurrency
Applying service 'autoscale-go' in namespace 'default':

  0.049s The Configuration is still working to reflect the latest desired specification.
  3.689s Traffic is not yet migrated to the latest revision.
  3.765s Ingress has not yet been reconciled.
  3.848s Waiting for load balancer to be ready
  4.386s Ready to serve.

Service 'autoscale-go' applied to latest revision 'autoscale-go-00024' is available at URL:
http://autoscale-go.default.kn.nima-dev.com
kn apply done
stopping threads...
Done.
starting threads
============ Experiment Started ============
Time Started: 2021-03-06 11:38:19.218540-05:00
Total Requests Made: 3703
stopping threads...
Done.
Experiment Name: autoscale_go_500_10k_5
Result

In [9]:
now = get_time_with_tz()
res_name = 'overview_' + now.strftime('res-%Y-%m-%d_%H-%M-%S')
res_file_name = res_name + ".csv"
res_folder = f'results/{workload_spec["exp_spec"]["name"]}/'
df = pd.DataFrame(data=overall_results)
df.to_csv(os.path.join(res_folder, res_file_name))

df

,target,rps,res_name,request_func,workload_name,is_rps,exp_time_mins
0,1,1,res-2021-03-06_12-39-03,autoscale_go_500_10k_5,autoscale-go,False,60
1,1,2,res-2021-03-06_13-39-50,autoscale_go_500_10k_5,autoscale-go,False,60
2,1,3,res-2021-03-06_14-40-25,autoscale_go_500_10k_5,autoscale-go,False,60
3,1,5,res-2021-03-06_15-40-49,autoscale_go_500_10k_5,autoscale-go,False,60
4,1,7,res-2021-03-06_16-41-11,autoscale_go_500_10k_5,autoscale-go,False,60
5,1,10,res-2021-03-06_17-41-30,autoscale_go_500_10k_5,autoscale-go,False,60
6,1,15,res-2021-03-06_18-41-47,autoscale_go_500_10k_5,autoscale-go,False,60
7,1,20,res-2021-03-06_19-42-03,autoscale_go_500_10k_5,autoscale-go,False,60
8,2,1,res-2021-03-06_20-43-15,autoscale_go_500_10k_5,autoscale-go,False,60
9,2,2,res-2021-03-06_21-44-01,autoscale_go_500_10k_5,autoscale-go,False,60
